# Mini Reto 008 del grupo Python para Trading

# Patrones estacionarios - seasonality

Hablando con un miembro del grupo, **Ignacio Villalonga**, sobre posible mini Retos, decidió presentar el siguiente mini reto a modo de introducción en el análisis de patrones estacionarios.  


La estacionalidad es algo que ha estado presente desde siempre en los mercados, ya que es una de las cosas que más condicionan el día a día de todo el mundo. 

Coges vacaciones en verano, descansas los fines de semana, recibes una nómina a fin de mes, en diciembre tienes que cuadrar las cuentas del año si quieres beneficiarte fiscalmente, en invierno hace más frio, comes fruta diferente y pones más calefacción. 

Y todo eso, impacta. En un mundo globalizado, variado y tan tecnológico, quizás estos impactos cada vez sean más leves, pero existen.

Y en concreto, existen en lo que el humano no puede controlar: la posición de la tierra respecto al sol cambia, y con ello cambian las cosechas. Y con las cosechas… pasan cosas.

https://kaufmansignals.com/capturing-seasonality-with-commodity-etfs/

https://commodityseasonality.com/grains

El objetivo a estudiar en este `Mini Reto`, es si existe una ventaja estacional en alguna materia prima.

Para ello, lo ideal sería encontrar una tabla que sea capaz de darte las rentabilidades anualizadas en función de distintos periodos.

Cada columna correspondería a un año, mientras que cada fila correspondería a un periodo dentro de ese año y dentro aparecería el retorno de ese activo para ese año y periodo.

    	2011	2012	2013	2014	2015
    CW1   X%				
    CW2					
    CW3					
    CW4					
(son semanas, pero bien podrían ser días)


Partiendo del código que hay a contianuación, **se pide**:

- Calcular la evolución de los rendimientos diarios durante un año, para cada uno de los años del CORN.
- Mostrar en una gráfica la evolución de rendimientos de cada uno de los años del CORN. De tal forma que en el eje X se vea el tiempo a lo largo de un año, y en el eje Y los % de retorno. Habiendo una curva por cada uno de los distintos años. 
- Calcular los retornos mensuales medios de cada año, para el CORN.
- Mostrar en una gráfica la evolución de los rendimientos de cada mes, para cada uno de los años del CORN.
- Mostrar una nueva gráfica de barras, con los retornos mensuales medios para los últimos 10 años.
- Repetir el proceso de calculo a 10 años y mostrar en gráficos de barras, pero en lugar de mensual, semanal.


La gráficas de retornos en lineas y barras deben quedar similar a la siguiente imagen:

![Analisis](Seasonality-analisys.png)




## Partir del siguiente código

In [1]:
import yfinance as yf

In [2]:
symbol = 'CORN'

In [3]:
data = yf.download(symbol, '2010-1-1','2020-1-1')

[*********************100%***********************]  1 of 1 completed


In [4]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-09,25.120001,25.250000,25.120001,25.150000,25.150000,1700
2010-06-10,25.459999,25.459999,25.459999,25.459999,25.459999,200
2010-06-11,25.879999,25.879999,25.790001,25.790001,25.790001,500
2010-06-14,25.990000,26.110001,25.990000,26.110001,26.110001,2200
2010-06-15,26.240000,26.240000,25.969999,25.969999,25.969999,7000
...,...,...,...,...,...,...
2019-12-24,14.830000,14.830000,14.780000,14.810000,14.810000,14700
2019-12-26,14.800000,14.840000,14.780000,14.840000,14.840000,28400
2019-12-27,14.850000,14.900000,14.820000,14.880000,14.880000,50500


### Vamos a definir algunas funiciones y variables que vamos a necesitar más adelante. Para generar gráficos interactivos vamos a usar pycharm, un wrapper de echarts para Python.

In [5]:
import pandas as pd
from datetime import datetime, timedelta
import pyecharts.options as opts
from pyecharts.charts import Line, Bar, Boxplot

def line_chart_returns(returns, show_notebook=True, is_symbol_show=True):
    _data = returns.mul(100).round(2)
    c = Line()
    c.add_xaxis(_data.index.astype(str).tolist())
    for col in _data:
        c.add_yaxis(col, _data[col].tolist(), is_connect_nones=True,
                    is_symbol_show=is_symbol_show,
                    label_opts=opts.LabelOpts(is_show=False))
    c.set_global_opts(tooltip_opts=opts.TooltipOpts(trigger="axis", is_show=True),
                      xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15),
                                               type_="category", boundary_gap=False),
                      yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value}%")),
                      )
    if show_notebook:
        display(c.render_notebook())
    return c

def bar_chart_rerturns(returns, show_notebook=True):
    _data = returns.mul(100).round(2)
    c = Bar()
    c.add_xaxis(_data.index)
    c.add_xaxis(_data.index.astype(str).tolist())
    val_max = _data.abs().max().max()
    for col in _data:
        c.add_yaxis(col, _data[col].tolist(),
                    label_opts=opts.LabelOpts(is_show=False))
    c.set_global_opts(yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value}%")),
                      tooltip_opts=opts.TooltipOpts(
                          trigger="axis", is_show=True),
                      xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-25),
                                               type_="category"),
                      visualmap_opts=opts.VisualMapOpts(min_=-val_max, max_=val_max,
                                                        range_color=[
                                                            '#E52B50', '#FFD500', '#00CC22'],
                                                        is_show=False))
    if show_notebook:
        display(c.render_notebook())
    return c

def box_chart_rerturns(returns, show_notebook=True):
    _data = returns.mul(100).round(2)
    c = Boxplot()
    c.add_xaxis(_data.index.tolist())
    c.add_yaxis('', c.prepare_data(_data.values.tolist()),
                itemstyle_opts=opts.ItemStyleOpts(color='#7CB9E8', border_color='#00308F', opacity=.8))
    c.set_global_opts(
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value}%")),
        xaxis_opts=opts.AxisOpts(
            splitarea_opts=opts.SplitAreaOpts(
                areastyle_opts=opts.AreaStyleOpts(opacity=1)),
            axislabel_opts=opts.LabelOpts(rotate=-45),
        ), tooltip_opts=opts.TooltipOpts(axis_pointer_type=False,
                          trigger="axis", is_show=True, is_show_content=True),)
    if show_notebook:
        display(c.render_notebook())
    return c

meses = ['Enero ',
         'Febrero ',
         'Marzo ',
         'Abril ',
         'Mayo ',
         'Junio ',
         'Julio ',
         'Agosto ',
         'Septiembre ',
         'Octubre ',
         'Noviembre ',
         'Diciembre ']

Calcular la evolución de los rendimientos diarios durante un año, para cada uno de los años del CORN.

In [6]:
returns = data['Close'].pct_change()
returns = returns.reindex(pd.date_range('2010-1-1','2019-12-31'))

In [7]:
daily_returns = returns.groupby([returns.index.strftime('%m-%d'), returns.index.year]).last().unstack().fillna(0)
daily_returns

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
01-02,0.000000,0.000000,0.000000,-0.014892,-0.001635,-0.006006,0.000000,0.000000,0.004770,0.004361
01-03,0.000000,-0.014099,0.015245,-0.005039,0.007534,0.000000,0.000000,0.011758,-0.000593,0.008064
01-04,0.000000,-0.014301,-0.002581,-0.015424,0.000000,0.000000,-0.016023,0.012678,-0.003563,0.009231
01-05,0.000000,0.015827,-0.023524,0.000000,0.000000,0.024169,0.001437,0.002608,-0.001192,0.000000
...,...,...,...,...,...,...,...,...,...,...
12-27,0.003141,0.018807,-0.002717,-0.003221,0.000000,0.000000,0.019925,0.003572,0.005632,0.002695
12-28,0.011221,0.010930,0.002952,0.000000,0.000000,-0.007411,-0.015312,-0.005338,0.001867,0.000000
12-29,-0.001806,-0.002883,0.000000,0.000000,-0.005413,0.002333,0.003753,0.000000,0.000000,0.000000
12-30,-0.008015,0.011566,0.000000,-0.007754,-0.014877,-0.009311,-0.000534,0.000000,0.000000,-0.004032


Mostrar en una gráfica la evolución de rendimientos de cada uno de los años del CORN. De tal forma que en el eje X se vea el tiempo a lo largo de un año, y en el eje Y los % de retorno. Habiendo una curva por cada uno de los distintos años. 

In [8]:
daily_year_equity = daily_returns.add(1).cumprod().sub(1)
line_chart_returns(daily_year_equity, is_symbol_show=False);

Calcular los retornos mensuales medios de cada año, para el CORN.

In [9]:
monthly_returns = data['Close'].groupby([data.index.year, data.index.month]).last().pct_change().unstack(0)
monthly_returns.index = meses
monthly_returns.style.format('{:,.2%}', na_rep="-")

Date,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Enero,-,5.10%,-2.62%,2.66%,1.05%,-6.23%,2.36%,2.30%,2.27%,0.87%
Febrero,-,6.61%,0.51%,-6.42%,5.11%,3.28%,-5.02%,1.41%,2.68%,-3.34%
Marzo,-,1.24%,-4.45%,-3.95%,6.71%,-4.11%,-2.47%,-1.96%,1.99%,-2.68%
Abril,-,4.50%,-0.94%,1.44%,1.64%,-4.93%,9.05%,-1.68%,1.28%,-1.05%
Mayo,-,-2.03%,-6.89%,2.36%,-10.39%,-3.78%,0.91%,1.39%,-0.88%,10.88%
Junio,-,-10.60%,16.27%,-8.95%,-6.78%,14.23%,-7.41%,0.42%,-8.82%,-1.62%
Julio,6.75%,10.52%,21.00%,-6.03%,-11.62%,-10.48%,-7.61%,-0.89%,3.16%,-3.53%
Agosto,6.43%,12.15%,0.00%,0.44%,-1.00%,-0.99%,-6.49%,-6.30%,-4.48%,-7.00%
Septiembre,10.52%,-20.56%,-4.95%,-8.03%,-11.53%,2.84%,5.48%,-0.28%,-2.04%,2.92%
Octubre,15.08%,8.35%,-0.58%,-3.66%,15.32%,-2.04%,4.07%,-2.44%,1.58%,-0.20%


In [22]:
monthly_returns.mean(1).to_frame('Retorno mensual medio').style.format('{:,.2%}')

,Retorno mensual medio
Enero,0.86%
Febrero,0.54%
Marzo,-1.08%
Abril,1.03%
Mayo,-0.94%
Junio,-1.47%
Julio,0.13%
Agosto,-0.72%
Septiembre,-2.56%
Octubre,3.55%


Mostrar en una gráfica la evolución de los rendimientos de cada mes, para cada uno de los años del CORN.

In [11]:
monthly_returns.index = meses
line_chart_returns(monthly_returns, is_symbol_show=True);

Mostrar una nueva gráfica de barras, con los retornos mensuales medios para los últimos 10 años.

In [12]:
avg_monthly_returns = monthly_returns.mean(1).to_frame('Retornos medios mensuales')

In [14]:
bar_chart_rerturns(avg_monthly_returns);

### Para mi es mucho más informativo visualizar la información de los retornos mensuales, o del periodo estudiado, con un gráfico de boxplot o de cajas y bigotes. Pues podemos analizar como se distribuye las muestras no solo su media.

In [16]:
box_chart_rerturns(monthly_returns.dropna(1));

Repetir el proceso de calculo a 10 años y mostrar en gráficos de barras, pero en lugar de mensual, semanal.

In [17]:
weekly_returns = data['Close'].groupby([data.index.year, data.index.isocalendar().week]).last().pct_change().unstack(0).iloc[:52]
weekly_returns.style.format('{:,.2%}', na_rep="-")

Date,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
week,,,,,,,,,,
1,-,-4.05%,5.57%,-30.77%,-13.93%,-4.44%,-0.38%,2.19%,-4.29%,-8.07%
2,-,7.48%,-11.57%,43.85%,16.97%,0.64%,0.76%,0.10%,3.49%,9.93%
3,-,1.17%,0.66%,2.32%,-1.70%,-2.93%,1.69%,2.66%,1.20%,0.49%
4,-,-1.47%,4.11%,-1.31%,0.42%,0.50%,0.28%,-1.83%,1.01%,-0.24%
5,-,5.11%,0.54%,1.71%,0.46%,-3.92%,-1.29%,0.67%,0.77%,-0.43%
6,-,3.53%,-2.76%,-4.05%,2.30%,3.72%,-1.91%,2.21%,0.23%,-1.05%
7,-,-0.32%,1.22%,-1.61%,-0.38%,0.42%,1.47%,-1.51%,1.17%,0.25%
8,-,-0.25%,-0.91%,-1.62%,1.24%,-0.58%,-2.39%,-1.02%,0.00%,0.56%
9,-,0.94%,2.14%,1.74%,1.88%,-0.27%,-0.24%,2.12%,2.36%,-2.72%


In [18]:
line_chart_returns(weekly_returns);

In [19]:
avg_weekly_returns = weekly_returns.mean(1).to_frame('Retornos Medios Semanales')
avg_weekly_returns.style.format('{:,.2%}', na_rep="-")

,Retornos Medios Semanales
week,
1,-6.46%
2,7.96%
3,0.62%
4,0.16%
5,0.40%
6,0.25%
7,0.08%
8,-0.55%
9,0.88%


In [20]:
bar_chart_rerturns(avg_weekly_returns);

In [21]:
c = box_chart_rerturns(weekly_returns.dropna(1));

# Vemos como la primera semana del año históricamente ha sido negativa en la mayoria de los casos y la segunada positiva, con portentajes absolutos superiores al resto del año.